In [2]:
# Load packages
import requests 
import pandas as pd 
import time
import re

from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [36]:
url = 'https://www.hellopeter.com/uber-eats?page='  
# urls = [url + str(page) for page in range(1, 4)]
# print(urls)

In [37]:
resp = requests.get(url)
content = BeautifulSoup(resp.content, 'lxml')
print(content)

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=no" name="viewport"/><title>Review Companies Online | Customer Service &amp; Company Ratings | hellopeter.com</title><meta content="https://www.hellopeter.com/static/img/logo/hp-fb-crawl-3.png" property="og:image"/><link href="/static/img/logo/favicon-180x180.png?1" rel="apple-touch-icon" sizes="180x180"/><link href="/static/img/logo/favicon-32x32.png?1" rel="icon" sizes="32x32" type="image/png"/><link href="/static/img/logo/favicon-16x16.png?1" rel="icon" sizes="16x16" type="image/png"/><link color="#25a8e2" href="/static/img/safari-pinned-tab.svg" rel="mask-icon"/><link crossorigin="anonymous" href="https://use.fontawesome.com/releases/v5.5.0/css/all.css" integrity="sha384-B4dIYHKNBt8Bc12p+WXckhzcICo0wtJAoU8YZTY5qE0Id1GSseTk6S+L3BlXeVIU" rel="stylesheet"/><link href="https://fonts.googleapis.com/css?family=Open+Sans:300,300i,400,400i,600,600i,70

In [7]:
def scrape_reviews(url):

  service = Service(executable_path="C:\\Program Files (x86)\\chromedriver.exe")
  driver = webdriver.Chrome(service=service)
  reviews = []

  driver.get(url)
    
  cards = driver.find_elements(By.CSS_SELECTOR, '.review-card')

  for card in cards:
  
    try:
      name = card.find_element(By.CSS_SELECTOR, '.has-text-weight-bold').text

      # Find date element using BeautifulSoup for more flexibility
      date_element = card.find_element(By.XPATH, ".//div[contains(text(), 'at')]")
      date_text = date_element.text

      # Extract date using BeautifulSoup and regex
      soup = BeautifulSoup(date_text, 'html.parser')
      date_match = re.search(r'(\d+ \w+ \d+ at \d+:\d+)', soup.get_text())
      date = date_match.group(1) if date_match else None
      
      rating = len(card.find_elements(By.CSS_SELECTOR, '.is-star-1'))
      
      title = card.find_element(By.CSS_SELECTOR, '.has-text-weight-bold.margin-top-24').text
      
      review = card.find_element(By.CSS_SELECTOR, '.margin-bottom-15').text
      
      review_data = {
        'review': review,
        'date': date, 
        'stars': rating,
        'name': name,
        'source': 'hellopeter',
        'title': title
      }

      reviews.append(review_data)
      
    except Exception as e:
      print(e)
      continue


  driver.quit()

  return pd.DataFrame(reviews)

In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import re
import pandas as pd
import time

def scrape_reviews(url):
  service = Service(executable_path="C:\\Program Files (x86)\\chromedriver.exe")

  driver = webdriver.Chrome(service=service)

  reviews = []

  driver.get(url)

  num_pages = 5

  for i in range(num_pages):

    # Wait for reviews to load
    time.sleep(5)

    cards = driver.find_elements(By.CSS_SELECTOR, '.review-card')

    for card in cards:

      try:
        name = card.find_element(By.CSS_SELECTOR, '.reviewer-name').text

        date_element = card.find_element(By.XPATH, ".//div[contains(text(), 'on')]")
        date_text = date_element.text

        soup = BeautifulSoup(date_text, 'html.parser')
        date = soup.select_one('div:nth-of-type(1)').text

        rating = len(card.find_elements(By.CSS_SELECTOR, '.fa-star.checked'))

        title = card.find_element(By.CSS_SELECTOR, '.review-title').text
        
        review = card.find_element(By.CSS_SELECTOR, '.review-content').text

        review_data = {
          'name': name,
          'date': date,
          'rating': rating,
          'title': title,
          'review': review
        }

        reviews.append(review_data)

      except Exception as e:
        print(e)
        continue

    # Click next button
    try:
      next_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, ".next-page"))
      )
      next_button.click() 

    except:
      print("No more pages")
      break
      
    time.sleep(5)
      
  driver.quit()

  return pd.DataFrame(reviews)

In [48]:
def scrape_reviews(start_url, num_pages=5):
    service = Service(executable_path="C:\\Program Files (x86)\\chromedriver.exe")
    driver = webdriver.Chrome(service=service)
    item_list = []

    current_page = 1
    while current_page <= num_pages:
        driver.get(start_url)
        content = driver.find_elements(By.CLASS_NAME, 'hp-card.review-card')

        for post in content:
            try:
                name = post.find_element(By.CLASS_NAME, 'font-size-16').text
                
                date_element = post.find_element(By.XPATH, ".//div[contains(text(), 'on')]")
                date_text = date_element.text

                soup = BeautifulSoup(date_text, 'html.parser')
                date = soup.select_one('div:nth-of-type(1)').text

                date = ' '.join(date)
                stars = post.find_element(By.CLASS_NAME, 'is-star-1').text.count('1')
                review = post.find_element(By.CLASS_NAME, 'margin-bottom-15').text

                data = {
                    'name': name,
                    'date': date,
                    'stars': stars,
                    'review': review,
                    'source': 'hellopeter',
                }
            except Exception as e:
                print(f"Error: {e}")
                continue

            item_list.append(data)

        try:
            # Scroll down to load more reviews
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # Add a delay to allow the page to load after scrolling

            # Find and click the "Next" button with waiting
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, 'pagination-next'))
            )
            next_button.click()
            current_page += 1

            # Add a delay to allow the page to load before scraping
            time.sleep(2)
        except Exception as e:
            print(f"Error clicking 'Next' button: {e}")
            break  # Break the loop if there is an issue clicking the next button

    driver.quit()
    return pd.DataFrame(item_list)

In [49]:
df_reviews = scrape_reviews(url, num_pages=5)
df_reviews

Error: 'NoneType' object has no attribute 'text'
Error: 'NoneType' object has no attribute 'text'
Error: 'NoneType' object has no attribute 'text'
Error: 'NoneType' object has no attribute 'text'
Error: 'NoneType' object has no attribute 'text'
Error: 'NoneType' object has no attribute 'text'
Error: 'NoneType' object has no attribute 'text'
Error: 'NoneType' object has no attribute 'text'
Error: 'NoneType' object has no attribute 'text'
Error: 'NoneType' object has no attribute 'text'
Error: 'NoneType' object has no attribute 'text'
Error clicking 'Next' button: Message: 



""


In [6]:
df_reviews.to_csv("ubereats_hellopeter.csv", index= False)
